In [17]:
import pandas as pd
import math
import os

In [18]:
class Subject:
    def __init__(self, name, descriptions):
        self.descriptions = descriptions
        self.name = name
        
class Description:
    def __init__(self, subject_name, university, description):
        self.name = subject_name
        self.university = university
        self.description = description
    
    def __to_dict__(self):
        return {
            'Subject': self.name,
            'University': self.university,
            'Description': self.description
        }
    
class V2L_Data:
    def __init__(self, paper, university, university_subject_name, topics, CO, CO_time, branch=None):
        self.paper = paper
        self.university = university
        self.university_subject_name = university_subject_name
        self.topics = int(topics) if not pd.isna(topics) else topics
        self.CO = int(CO) if not pd.isna(CO) else CO
        self.CO_time = int(CO_time) if not pd.isna(CO_time) else CO_time
        self.branch = branch

In [32]:
def get_subjects(df, is_diploma=False):
    a = df.loc[df['S.N'].notnull()]
    subj_name = 'Subject Name' if not is_diploma else 'Subject Name_ES'
    
    subjects = {}
    subject_cutoffs = []
    for idx, row in a.iterrows():
        subject_cutoffs.append(idx)
    
    #print(a)
    #print(subject_cutoffs)
    for i in subject_cutoffs:
        subjects[i] = df.iloc[i][subj_name]
    
    return subject_cutoffs,subjects

def extract_info(df, is_diploma=False):
    indices,subjects = get_subjects(df, is_diploma)
    
    if is_diploma:
        subj_name, univ_name, univ_subj_name = 'Subject Name_ES', 'BTE short name', 'Subject NameBTE'
    else:
        subj_name, univ_name, univ_subj_name = 'Subject Name', 'University short name', 'University Subject Name'
    
    result = {}
    pws = {}
    for i,subject in enumerate(subjects.keys()):
        name = subjects[subject].split("(")[0].strip()
        password = df[df[subj_name]==subjects[subject]].Password.item()
        if i == len(subjects)-1:
            val = df[indices[i]:]

        else:
            val = df[indices[i]:indices[i+1]]
        
        data_rows = {}
        for value,g in val.groupby([univ_name]):
            university = value
            data = []
            prev = None
            for x, row in enumerate(g.iterrows()):
                row = row[1]
                    
                if prev != None:
                    if prev.topics == row['# Topics'] and prev.CO == row['# CLOs']:
                        # Same paper code as before
                        paper = prev.paper + "/" + row['Paper code']
                        data[-1].paper = paper
                        continue

                print(name, university, row['Paper code'], row['# CLOs'])
                d = V2L_Data(row['Paper code'], university, row[univ_subj_name], row['# Topics'], row['# CLOs'], row['CLO time'])            
                prev = d
                data.append(d)
            
            data_rows[university] = data
        
        
        result[name] = {'data':data_rows, 'password': password}
    
    return result

In [28]:
def get_tabby_data_v2l(university_data):
    final_tabby = ''
    #draft = '[tabby title={0}]\n\n{1} offers this course for <strong><u>{2}</u></strong> branches.\n<ul><li><strong>{3} Video Learning Outcomes</strong> attained across<strong> {4} topics</strong> taught over <strong>{5} hours</strong> of instruction.</li><li>Total duration of course: <strong>{5} hours</strong></li></ul>'
    draft = '[tabby title={0}]\n\n{1} offers this course under the name {2}.\n<ul><li><strong>{3} Video Learning Outcomes</strong> attained across<strong> {4} topics</strong> taught over <strong>{5} hours</strong> of instruction.</li><li>Total duration of course: <strong>{5} hours</strong></li></ul>'

    for univ in university_data:
        d = draft.format(univ.paper, univ.university.upper(), univ.university_subject_name.capitalize(), univ.CO, univ.topics, univ.CO_time)
        final_tabby = final_tabby + d
    
    final_tabby = final_tabby + '[tabbyending]\n\n'
    
    return final_tabby

def generate_description(university_data, university, subject, password, v2l=True, is_diploma=False):    
    button = '''<button class=\"single_add_to_cart_button button alt\" style=\"font-family: \'Open Sans\'; font-size: .8em;\" onclick=\"window.open(\'https://go.2learn.in/login/index.php\', \'_blank\'); return false;\">View Sample Course</button>\n <strong>Username</strong>: <i>{0}</i>; <strong>Password</strong>: <i>{1}</i>\n\n'''
    if is_diploma:
        top = '''<strong>{1}</strong> paper code(s) are comprehensively covered in <strong>{0}</strong> course.\nCheck out your complete syllabus coverage in <a href="https://cc.2learn.in/det" target="_blank">Curriculum Coverage Plan (CCP)</a>.\n\n'''
    else:
        top = '''<strong>{1}</strong> paper code(s) are comprehensively covered in <strong>{0}</strong> course.\nCheck out your complete syllabus coverage in <a href="https://cc.2learn.in/btech" target="_blank">Curriculum Coverage Plan (CCP)</a>.\n\n'''
    #end = 'Vdo2learn is a shared-library resource requiring no user interaction. Courses are organised by Units, Sections and Topics. Each Topic consists of one or more Video Learning Modules (VLM). Each VLM is taught through several Concept Learning Outcomes (CLO). A CLO is a statement of what a student should expect to recall, solve or perform upon its completion.\n\nCourse Faculty has 10+ years of teaching and industry experience. The lessons are taught systematically, from the very basics, and in simple, lucid language and manner. The Course is designed to inculcate a clear understanding of subject-concepts and enable students to gain theoretical as well as practical insights.\n'
    end = '''V2L courses enjoy a finely granulated hierarchy organised by Units, Sections, Topics, Video Learning Modules (VLM) and Concept Learning Outcomes (CLO).\n\nA CLO is a statement of what a student should expect to recall, solve or perform upon its completion.\n\nCourse Faculty carry several years of teaching and industry experience. The lessons are taught systematically, and delivered in simple, lucid language and manner. Courses are designed to inculcate a clear understanding of subject-concepts and enable students to gain theoretical and practical insights.\n'''
    final_result = button.format(password, password)
    
    name = university.upper() + "-" + subject
    papers = ", ".join([y for x in university_data for y in x.paper.split("/")])
    papers = " and".join(papers.rsplit(",", 1))
    top = top.format(name, papers)
    
    if v2l:
        tabby = get_tabby_data_v2l(university_data)
        top = top + tabby
        
        
    final_result += top
    final_result += end
    return final_result
        

In [29]:
def get_descriptions(csv_folder, out_folder='descriptions_new/', is_diploma=False):
    
    for idx, file in enumerate(os.listdir(csv_folder)):

        csv_file = os.path.join(csv_folder, file)
        df = pd.read_csv(csv_file)
        data = extract_info(df, is_diploma)

        for i, subject in enumerate(data.keys()):
            descriptions = []
            university_data = data[subject]['data']
            password= data[subject]['password']
            for university in university_data.keys():
                description = generate_description(university_data[university], university, subject, password, is_diploma=is_diploma)
                descriptions.append(Description(subject, university, description))

            d = pd.DataFrame([dd.__to_dict__() for dd in descriptions])
            d.to_csv(out_folder + subject + ".csv",index=False)
            if idx==0 and i==0:
                d.to_csv(out_folder + "combined.csv", mode='a', index=False)
            else:
                d.to_csv(out_folder + "combined.csv", mode='a', header=False, index=False)



#### Code for generating Btech CD

In [33]:
csv_dir = "files_new/btech"
get_descriptions(csv_dir, 'descriptions_new/btech/')

Engineering Mechanics AKTU KME402 126
Engineering Mechanics BPUT REM2B001 54
Engineering Mechanics DBATU ME202 78
Engineering Mechanics GGSIPU ETME-110 68
Engineering Mechanics GTU 3132005 118
Engineering Mechanics JCBUST ESC-202 79
Engineering Mechanics MAKAUT ES-AUE301 79
Engineering Mechanics MDU ESC-ME- 209G 54
Engineering Mechanics MRSPTU BMECE0-001 80
Engineering Mechanics MU FEC104 56
Engineering Mechanics RTMNU BES2-7T 52
Engineering Mechanics RTU 3AE3-04 83
Engineering Mechanics SPPU 101011 66
Numerical Methods DBATU BTBSC406 65
Numerical Methods DBATU BTCOE405-A 69
Numerical Methods DBATU BTCVE404A 71
Numerical Methods DBATU BTMEC404 65
Numerical Methods GGSIPU ETMA-203 69
Numerical Methods GTU 2140606 90
Numerical Methods GTU 2140706 65
Numerical Methods MAKAUT MCS-301 69
Numerical Methods MRSPTU BCIE1-627 58
Engineering Math 1 AKTU KAS103 134
Engineering Math 1 BPUT RMA1A001 97
Engineering Math 1 CSVTU 300114(14) 81
Engineering Math 1 DBATU BTBS101 107
Engineering Math 1 GG

Soil Mechanics MU CEC601 77
Soil Mechanics RGPV CE604 136
Soil Mechanics RGPV CE801 63
Soil Mechanics RTMNU BECVE-402T 96
Soil Mechanics RTMNU BECVE504T 68
Soil Mechanics RTU 5CE4-04 158
Soil Mechanics SPPU 201008 115
Soil Mechanics SPPU 301015-d 65
Soil Mechanics SPPU 301009 48
Soil Mechanics UTU BCET-502 112
Soil Mechanics UTU BCET-604  51
Building Material and Construction AKTU KCE401 128
Building Material and Construction CSVTU 320355(20) 44
Building Material and Construction DBATU BTCVC305 60
Building Material and Construction GGSIPU ETCE-207 90
Building Material and Construction GTU 140602 64
Building Material and Construction IKGPTU BTCE305 97
Building Material and Construction MAKAUT CE303 106
Building Material and Construction MRSPTU BCIE1-304 94
Building Material and Construction MU CEC-404 80
Building Material and Construction RGPV CE302 89
Building Material and Construction RTMNU BECVE-405T 62
Building Material and Construction RTU 3CE4-07 94
Building Material and Construct

Electrical Measurements and Measuring Instruments RTMNU BEENE303T 91.0
Electrical Measurements and Measuring Instruments RTMNU BEELE303T 112.0
Electrical Measurements and Measuring Instruments RTU 4EE3-04 82.0
Electrical Measurements and Measuring Instruments RTU 4EC3-06 52.0
Electrical Measurements and Measuring Instruments RTU 4EI3-06 98.0
Electrical Measurements and Measuring Instruments SPPU 203144 111.0
Electrical Measurements and Measuring Instruments SPPU 206263 87.0
Electrical Measurements and Measuring Instruments UTU BECT 302 77.0
Electrical Measurements and Measuring Instruments UTU BEET 301 104.0
Analog and Digital Communication System AKTU KEC401 205.0
Analog and Digital Communication System AKTU REE052 145.0
Analog and Digital Communication System AKTU REC602 133.0
Analog and Digital Communication System BPUT REE6C001 108.0
Analog and Digital Communication System BPUT PC 11 223.0
Analog and Digital Communication System CSVTU 312553(12) 124.0
Analog and Digital Communicati

#### Code for generating Diploma CD

In [10]:
csv_dir = "files/v2l-diploma"
get_descriptions(csv_dir, 'descriptions_new/diploma/', True)

Applied Math 1 DLBTE 79
Applied Math 1 GJBTE 81
Applied Math 1 HRBTE 125
Applied Math 1 MHBTE 54
Applied Math 1 MPBTE 99
Applied Math 1 PBBTE 99
Applied Math 1 RJBTE 137
Applied Math 1 UPBTE 89
Applied Math 1 WBBTE 103
Applied Math 2 DLBTE 40
Applied Math 2 GJBTE 35
Applied Math 2 MHBTE 56
Applied Math 2 MPBTE 53
Applied Math 2 PBBTE 52
Applied Math 2 UPBTE 51
Applied Math 2 WBBTE 85
Applied Math 3 UPBTE 128
RCC DLBTE 69
RCC HRBTE 62
RCC MPBTE 69
RCC PBBTE 67
RCC RJBTE 71
RCC UPBTE 62
RCC WBBTE 69
Soil Mechanics DLBTE 107
Soil Mechanics GJBTE 84
Soil Mechanics HRBTE 107
Soil Mechanics MHBTE 112
Soil Mechanics MPBTE 119
Soil Mechanics PBBTE 107
Soil Mechanics RJBTE 107
Soil Mechanics UPBTE 107
Soil Mechanics WBBTE 115
Surveying DLBTE 52
Surveying DLBTE 37
Surveying GJBTE 72
Surveying GJBTE 43
Surveying HRBTE 59
Surveying HRBTE 56
Surveying MHBTE 57
Surveying MPBTE 61
Surveying MPBTE 48
Surveying PBBTE 59
Surveying PBBTE 56
Surveying RJBTE 61
Surveying RJBTE 43
Surveying UPBTE 59
Surveyi

Basic Electrical Engineering UPBTE 124
Basic Electrical Engineering UPBTE 92
Basic Electrical Engineering UPBTE 142
Basic Electrical Engineering WBBTE 94
Strength of Material DLBTE 106
Strength of Material DLBTE 110
Strength of Material GJBTE 103
Strength of Material GJBTE 76
Strength of Material HRBTE 110
Strength of Material HRBTE 93
Strength of Material MHBTE 84
Strength of Material MHBTE 79
Strength of Material MPBTE 126
Strength of Material MPBTE 92
Strength of Material PBBTE 100
Strength of Material PBBTE 89
Strength of Material RJBTE 142
Strength of Material RJBTE 123
Strength of Material UPBTE 105
Strength of Material UPBTE 94
Strength of Material WBBTE 74
Strength of Material WBBTE 54
Fluid Mechanics DLBTE 95
Fluid Mechanics DLBTE 81
Fluid Mechanics GJBTE 89
Fluid Mechanics GJBTE 64
Fluid Mechanics HRBTE 88
Fluid Mechanics HRBTE 81
Fluid Mechanics MHBTE 107
Fluid Mechanics MHBTE 101
Fluid Mechanics MPBTE 111
Fluid Mechanics MPBTE 78
Fluid Mechanics PBBTE 96
Fluid Mechanics PBB

In [252]:
a = "sdf sdf (sfd)".split("(")[0].strip()
df.loc[df['Subject Name']=='Data Structures', 'S.N'][0]

import os
for file in os.listdir('files'):
    print(file)

List of Subjects B.Tech - CC.csv
List of Subjects B.Tech - CE.csv
List of Subjects B.Tech - CS.csv
List of Subjects B.Tech - EX.csv
List of Subjects B.Tech - ME.csv
